**写在前面：**本节内容是 [Datawhale四月组队学习 - 集成学习（中）- CH4-集成学习之boosting -【Task10 前向分布算法与梯度提升决策树】](https://github.com/datawhalechina/team-learning-data-mining/tree/master/EnsembleLearning) 的学习笔记，对应CH4的第4、5节，学习周期3天

## 库

In [34]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1,make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

## sklaern参数
**参考资料：**
- [scikit-learn 梯度提升树(GBDT)调参小结](https://www.cnblogs.com/pinard/p/6143927.html)
- [机器学习系列(11)_Python中Gradient Boosting Machine(GBM）调参方法详解](https://blog.csdn.net/han_xiaoyang/article/details/52663170)

**GradientBoostingClassifier重要参数**：
- `loss`:{'deviance','exponential'}, default='deviance'  
    deviance:对数似然损失函数，对具有概率输出的分类的偏差；exponential:指数损失函数，退化为AdaBoost算法，只能用于二分类问题。
- `learning_rate`:float, default=0.1  
    学习率，即每个弱学习器的权重缩减系数α。
- `n_estimators`:int, default=100  
    弱学习器的个数，或者弱学习器的最大迭代次数。梯度提升对于过度拟合具有相当强的鲁棒性，因此大量提升通常会带来更好的性能。
- `subsample`:float, default=1.0  
    对每个学习器的子采样比例，可防止过拟合，选择<1的值会使方差减小偏差增大。
- `criterion`:{'friedman_mse', 'mse', 'mae'}, default='friedman_mse'  
    衡量划分质量的指标。'friedman_mse'通常是最好的，'mae'不推荐使用。
- `min_samples_split`:int or float, default=2  
    内部节点再划分所需最小样本数。可避免过拟合，但需要综合考虑样本数进行调整，避免出现欠拟合。
- `min_samples_leaf`:int or float, default=1  
    叶子节点最少样本数。可避免过拟合，但需要综合考虑样本数进行调整，避免出现欠拟合。需要注意的是，在不均等分类问题中(imbalanced class problems)，一般这个参数需要被设定为较小的值，因为大部分少数类别（minority class）含有的样本都比较小。
- `min_weight_fraction_leaf`:float, default=0.0  
    和`min_samples_leaf`相似，这里设置的是样本比例，和`min_samples_leaf`只需设置一个。
- `max_depth`:int, default=3  
    树的最大深度。 
- `min_impurity_decrease`:float, default=0.0  
    限制信息增益的大小。
- `min_impurity_split`:float, default=None  
    节点划分最小不纯度。(sklearn:不推荐使用，使用`min_impurity_decrease`代替)
- `init`:estimator or 'zero', default=None  
    一个estimator，用于计算初始预测。init必须提供fit和predict_proba
- `random_state`:int, RandomState instance or None, default=None  
    随机种子。
- `max_features`:{'auto', 'sqrt', 'log2'}, int or float, default=None  
    划分时考虑的最大特征数。'None'代表考虑所有特征；'log2'代表最多考虑$\log_2N$个特征；'sqrt'和'auto'代表最多考虑$\sqrt{N}$个特征；int代表考虑max_features个特征；float代表考虑int(max_features * n_features)个特征。会使方差减小和偏差增大。  
    - 注意：在找到至少一个有效的节点样本分区之前，分割的搜索不会停止，即使它需要有效检查多个max_features要素也是如此。
- `max_leaf_nodes`:int, default=None  
    最大叶子节点数。
- `warm_start`:bool, default=False  
    是否热启动。
- `validation_fraction`:float, default=0.1  
    判断早停时划分为验证集的样本比例。
- `n_iter_no_change`:int, default=None  
    迭代`n_iter_no_change`次中`tol`没有得到改善时，将会早停。
- `tol`:float, default=1e-4  
    早停的容忍度。当`n_iter_no_change`次迭代中不能得到`tol`的改善时，将会早停。
- `ccp_alpha`:non-negative float, default=0.0  
    Complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than `ccp_alpha` will be chosen. By default, no pruning is performed. See Minimal Cost-Complexity Pruning for details.  
    
    
**GradientBoostingRegressor重要参数**：  
- `loss`:{'ls', 'lad', 'huber', 'quantile'}, default='ls'  
    'ls'指最小二乘回归，'lad'指最小绝对偏差，'huber'是二者结合，'quantile'允许分位数回归（用于`alpha`指定分位数）。
- `learning_rate`:float, default=0.1  
    学习率。
- `n_estimators`:int, default=100  
    弱学习器的个数，或者弱学习器的最大迭代次数。梯度提升对于过度拟合具有相当强的鲁棒性，因此大量提升通常会带来更好的性能。
- `subsample`:float, default=1.0  
    对每个学习器的子采样比例，可防止过拟合，选择<1的值会使方差减小偏差增大。
- `criterion`:{'friedman_mse', 'mse', 'mae'}, default='friedman_mse'  
    衡量划分质量的指标。'friedman_mse'通常是最好的，'mae'不推荐使用。
- `min_samples_split`:int or float, default=2  
    内部节点再划分所需最小样本数。可避免过拟合，但需要综合考虑样本数进行调整，避免出现欠拟合。
- `min_samples_leaf`:int or float, default=1  
    叶子节点最少样本数。可避免过拟合，但需要综合考虑样本数进行调整，避免出现欠拟合。
- `min_weight_fraction_leaf`:float, default=0.0  
- `max_depth`:int, default=3  
    树的最大深度。 
- `min_impurity_decrease`:float, default=0.0  
    限制信息增益的大小。
- `init`:estimator or 'zero', default=None  
    一个estimator，用于计算初始预测。init必须提供fit和predict_proba
- `random_state`:int, RandomState instance or None, default=None  
    随机种子。
- `max_features`:{'auto', 'sqrt', 'log2'}, int or float, default=None  
    划分时考虑的最大特征数。'None'代表考虑所有特征；'log2'代表最多考虑$\log_2N$个特征；'sqrt'和'auto'代表最多考虑$\sqrt{N}$个特征；int代表考虑max_features个特征；float代表考虑int(max_features * n_features)个特征。会使方差减小和偏差增大。  
    - 注意：在找到至少一个有效的节点样本分区之前，分割的搜索不会停止，即使它需要有效检查多个max_features要素也是如此。
- `alpha`:float, default=0.9  
    Huber损失函数和分位数损失函数的alpha分位数。仅当loss='huber'或loss='quantile'时。
- `max_leaf_nodes`:int, default=None  
    最大叶子节点数。
- `warm_start`:bool, default=False  
    是否热启动。
- `validation_fraction`:float, default=0.1  
    判断早停时划分为验证集的样本比例。
- `n_iter_no_change`:int, default=None  
    迭代`n_iter_no_change`次中`tol`没有得到改善时，将会早停。
- `tol`:float, default=1e-4  
    早停的容忍度。当`n_iter_no_change`次迭代中不能得到`tol`的改善时，将会早停。
- `ccp_alpha`:non-negative float, default=0.0  
    Complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than `ccp_alpha` will be chosen. By default, no pruning is performed. See Minimal Cost-Complexity Pruning for details.  

## 一些尝试

In [32]:
X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
    max_depth=2, random_state=0, loss='ls').fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))

3.702628537100702

In [42]:
X, y = make_regression(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.440031029624667